## Imports by Country over Time Line Chart

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import plotly_express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc, callback, Output, Input

In [11]:
df = pd.read_csv('cleaned_data/stacked_data.csv')
df = pd.DataFrame(df)
df

,category,subcategory,country,year,value
0,Animals,"Bovine animals, live",Canada,2023,1311.1
1,Animals,"Bovine animals, live",Canada,2022,1132.0
2,Animals,"Bovine animals, live",Canada,2021,859.2
3,Animals,"Bovine animals, live",Canada,2020,872.7
4,Animals,"Bovine animals, live",Canada,2019,956.8
...,...,...,...,...,...
14445,Vegetables,Total,Spain,2019,254.9
14446,Vegetables,Total,Spain,2020,231.3
14447,Vegetables,Total,Spain,2021,240.8
14448,Vegetables,Total,Spain,2022,280.7


In [2]:
stacked_data = pd.read_csv('cleaned_data/stacked_data.csv')
stacked_data = pd.DataFrame(stacked_data)
stacked_data

,category,subcategory,country,year,value
0,Animals,"Bovine animals, live",Canada,2023,1311.1
1,Animals,"Bovine animals, live",Canada,2022,1132.0
2,Animals,"Bovine animals, live",Canada,2021,859.2
3,Animals,"Bovine animals, live",Canada,2020,872.7
4,Animals,"Bovine animals, live",Canada,2019,956.8
...,...,...,...,...,...
14445,Vegetables,Total,Spain,2019,254.9
14446,Vegetables,Total,Spain,2020,231.3
14447,Vegetables,Total,Spain,2021,240.8
14448,Vegetables,Total,Spain,2022,280.7


In [4]:
stacked_data[(stacked_data["subcategory"] == "Total")&(stacked_data['country']=="Canada")&(stacked_data['category']=="Vegetables")]

,category,subcategory,country,year,value
14200,Vegetables,Total,Canada,1999,795.5
14201,Vegetables,Total,Canada,2000,946.0
14202,Vegetables,Total,Canada,2001,1047.6
14203,Vegetables,Total,Canada,2002,1155.2
14204,Vegetables,Total,Canada,2003,1341.7
14205,Vegetables,Total,Canada,2004,1486.0
14206,Vegetables,Total,Canada,2005,1506.5
14207,Vegetables,Total,Canada,2006,1665.4
14208,Vegetables,Total,Canada,2007,1731.1
14209,Vegetables,Total,Canada,2008,1916.3


In [ ]:
app = dash.Dash(__name__)

app.layout = dash.html.Div([
    dash.html.H1("U.S. International Food Imports"),
    dash.dcc.Graph(id="line-chart"),
])

@app.callback(
    dash.dependencies.Output("line-chart", "figure"),
    dash.dependencies.Input("line-chart", "id")  # Input is necessary to trigger the callback
)
def update_chart(_):
    filtered_data = stacked_data[
        (stacked_data["subcategory"] == "Total") & 
        (stacked_data['country'].isin(["Canada","Mexico","Taiwan"])) & 
        (stacked_data['category'].isin(["Vegetables","Animals","Beverages"]))
    ]

    import_by_country_over_time_fig = px.line(
        filtered_data,
        x="year",
        y="value",
        color="country",
        line_dash="category",
        # markers=True,
        title="Imports by Country over Time",
        labels={"value": "Value (Million Dollars)", "year": "Year"}
    )

    return import_by_country_over_time_fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)

### Plot with DropDown & Timeline

In [18]:
# Initialize the app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1(
        children='Comparison of Imports Across Countries and Sub-Categories',
        style={
        'color': 'black',           # Text color
        'backgroundColor': 'white',  # Background color
        'textAlign': 'center',     # Center-align text
        'fontSize': '24px',        # Font size
        'padding': '10px'          # Padding around the text
        }),
    
    
    # Dropdown for countries
    dcc.Dropdown(
        options=[{'label': c, 'value': c} for c in df['country'].unique()],
        value=['Canada'],  # Default selection
        id='dropdown-selection-country',
        multi=True,  # Enable multiple selection
        placeholder="Select one or more countries"
    ),
    
    # Dropdown for categories
    dcc.Dropdown(
        options=[{'label': c, 'value': c} for c in df['category'].unique()],
        value=['Animals'],  # Default selection
        id='dropdown-selection-category',
        multi=True,  # Enable multiple selection
        placeholder="Select one or more categories"
    ),
    
    # Time slider for years
    dcc.Slider(
        min=df['year'].min(),
        max=df['year'].max(),
        step=1,
        marks={year: str(year) for year in range(df['year'].min(), df['year'].max() + 1)},
        value=df['year'].max(),  # Default to the most recent year
        id='slider-year-selection'
    ),

    # Graph
    dcc.Graph(id='line-chart'),
    
])

# Callback to update tabs and graphs based on selections
@callback(
    Output('line-chart', 'figure'),
    [
        Input('dropdown-selection-country', 'value'),
        Input('dropdown-selection-category', 'value'),
        Input('slider-year-selection', 'value'),
    ]
)

def update_chart(selected_countries, selected_categories, selected_year):
    filtered_data = df[
        (df["subcategory"] == "Total") & 
        (df['country'].isin(selected_countries)) & 
        (df['category'].isin(selected_categories))
    ]

    import_by_country_over_time_fig = px.line(
        filtered_data,
        x="year",
        y="value",
        color="country",
        line_dash="category",
        # markers=True,
        title="Imports by Country over Time",
        labels={"value": "Value (Million Dollars)", "year": "Year"}
    )

    return import_by_country_over_time_fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)